In [1]:
import pickle as pkl


In [2]:
with open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gpt/wikitables/wikitable_4o_output.pkl", "rb") as f:
    pred_tables = pkl.load(f)
    

In [6]:
pred_tables

['Final Answer: | title | portuguese heritage society | <NEWLINE> | subtitle | other activities | <NEWLINE> | name | michelle schimel | <NEWLINE> | office | new york state assemblywoman |',
 'Final Answer: | name | southwest missouri state | <NEWLINE> | years | 1982-1984 |',
 'Final Answer: | name | sonia gandhi | <NEWLINE> | award | order of king leopold | <NEWLINE> | year | 2006 | <NEWLINE> | awarded by | government of belgium |',
 'Final Answer: | name | emmanuel jal | <NEWLINE> | role | tulu | <NEWLINE> | film | africa united | <NEWLINE> | year | 2010 |',
 'Final Answer: | team | mississippi state bulldogs baseball | <NEWLINE> | opponent | virginia | <NEWLINE> | year | 2013 | <NEWLINE> | location | charlottesville, va |',
 "Final Answer: | name | jeremy bujakowski | <NEWLINE> | age | 28 | <NEWLINE> | event | men's downhill | <NEWLINE> | position | 53 |",
 'Final Answer: | title | minneapolis–saint paul international airport | <NEWLINE> | subtitle | passenger statistics | <NEWLINE> 

In [4]:
import json
import re
import pandas as pd

def extract_table(raw_text):
    """
    Extracts a table from a raw text string which may be in one of several formats:
    - Plain text with code fences (e.g., starting with ```text and ending with ```)
    - A JSON string that contains a "result" key with a table string, possibly wrapped in triple quotes.
    
    The table rows are expected to be separated by the literal "<NEWLINE>" token,
    and each row is a pipe-separated list of fields.
    """
    text = raw_text.replace("Final Answer:", "").strip()
    
    # --- Step 1. If the input is JSON, extract the "result" field ---
    try:
        parsed = json.loads(text)
        if isinstance(parsed, dict) and "result" in parsed:
            text = parsed["result"]
    except json.JSONDecodeError:
        # Not valid JSON, so assume it's a plain string
        pass

    # --- Step 2. Remove markdown code fences if present ---
    # Remove starting fence like ```text (or any language) and the ending ```
    text = re.sub(r'^```(?:[^\n]*)\n', '', text)
    text = re.sub(r'\n```$', '', text)

    # --- Step 3. Remove extra triple quotes if present ---
    text = text.strip()
    if text.startswith('"""') and text.endswith('"""'):
        text = text[3:-3].strip()

    # --- Step 4. Split the string into rows using the <NEWLINE> token ---
    if "<NEWLINE>" in text:
        rows = text.split("<NEWLINE>")
    else:
        rows = text.split("\n")
    records = []
    for row in rows:
        row = row.strip()
        if not row:
            continue
        # Remove any leading/trailing pipes and split on "|"
        # Then filter out any empty strings after stripping whitespace
        parts = [p.strip() for p in row.strip('|').split("|") if p.strip()]
        if len(parts) >= 2:
            key = parts[0]
            # In case there are additional tokens, join them with " | " to preserve inner separators.
            value = " | ".join(parts[1:])
            records.append((key, value))
    
    # --- Step 5. Create a DataFrame from the records ---
    df = pd.DataFrame(records, columns=["Field", "Value"])
    return df

# --- Example 1: Table wrapped in a markdown code fence ---
raw_data2 = """```text\n| name | john \"jack\" reynolds | <NEWLINE> | birth date | 21 february 1869 | <NEWLINE> | death date | 12 march 1917 | <NEWLINE> | clubs | west bromwich albion, aston villa, celtic, southampton | <NEWLINE> | international appearances | ireland (5), england (8) | <NEWLINE> | honours | FA Cup (1892 with West Bromwich Albion), English League titles (3 with Aston Villa), FA Cups (2 with Aston Villa) | <NEWLINE> | death place | sheffield |\n```\n"""
# --- Example 2: JSON string containing the table in the "result" key ---
raw_data1 = """| death place | sheffield, england | <NEWLINE> | name | jack reynolds | <NEWLINE> | death date | 12 march 1917 | <NEWLINE> | birth date | 21 february 1869 | <NEWLINE> | fullname | john reynolds |"""

# --- Parse and display the tables ---
df1 = extract_table(raw_data1)
df2 = extract_table(raw_data2)

print("Table from Example 1:")
print(df1.to_markdown(index=False))
print("\nTable from Example 2:")
print(df2.to_markdown(index=False))


Table from Example 1:
| Field       | Value              |
|:------------|:-------------------|
| death place | sheffield, england |
| name        | jack reynolds      |
| death date  | 12 march 1917      |
| birth date  | 21 february 1869   |
| fullname    | john reynolds      |

Table from Example 2:
| Field                     | Value                                                                                                             |
|:--------------------------|:------------------------------------------------------------------------------------------------------------------|
| name                      | john "jack" reynolds                                                                                              |
| birth date                | 21 february 1869                                                                                                  |
| death date                | 12 march 1917                                                                     

In [7]:
with open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gemini/wikitables/test.data", "r") as f: 
    tables= f.readlines()

In [9]:
id_to_table = {}

for i, table in enumerate(tables):
    id_to_table[str(i)] = extract_table(table.strip()).to_markdown(index= False)

In [10]:
input_tables_for_score = {}
for i, table in enumerate(pred_tables):
    input_tables_for_score[str(i)] = [id_to_table[str(i)], extract_table(table.strip()).to_markdown(index= False)]

In [11]:
input_tables_for_score

{'0': ['| Field    | Value                        |\n|:---------|:-----------------------------|\n| title    | portuguese heritage society  |\n| subtitle | other activities             |\n| name     | michelle schimel             |\n| office   | new york state assemblywoman |',
  '| Field    | Value                        |\n|:---------|:-----------------------------|\n| title    | portuguese heritage society  |\n| subtitle | other activities             |\n| name     | michelle schimel             |\n| office   | new york state assemblywoman |'],
 '1': ['| Field    | Value                    |\n|:---------|:-------------------------|\n| subtitle | football                 |\n| team     | southwest missouri state |\n| season   | 1982-1984                |',
  '| Field   | Value                    |\n|:--------|:-------------------------|\n| name    | southwest missouri state |\n| years   | 1982-1984                |'],
 '2': ['| Field                 | Value                   |\n|:----

In [12]:
json.dump(input_tables_for_score, open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gpt/wikitables/input_tables_for_score.json", "w"))